In [1]:
import transcriptic 
from transcriptic import commands, Container
from transcriptic.config import Connection
from transcriptic.jupyter import objects
import json
from pysd2cat.data import tx_fcs
from pysd2cat.data import pipeline
from pysd2cat.analysis import biofab_live_dead_analysis as blda
from pysd2cat.data import biofab_live_dead as bld


from os.path import expanduser
import requests
import os
import zipfile
import shutil
import fnmatch
import glob
import pandas as pd
import ast

%load_ext autoreload
%reload_ext autoreload
%autoreload 2




#run_id='r1dd37mcxv5pf4'
#run_id='r1dk8xp9dymm54' # Time series
#run_id='r1dmsrursbqwuz'
run_id = 'r1dnj926jr7q9g'

aliquot_map_technique = {
    'r1dd37mcxv5pf4' : "titration1",
    'r1dk8xp9dymm54' : "fifteen_zero",
    'r1dmsrursbqwuz' : "10to80",
    'r1dnj926jr7q9g' : "10to80_eth_wash"
}

work_dir='data/transcriptic/'+run_id
Connection.from_file("~/.transcriptic")
tx_config = json.load(open(os.path.join(expanduser("~"), ".transcriptic")))



if not os.path.exists(work_dir):
    os.makedirs(work_dir)

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/src/flowcytometrytools/FlowCytometryTools/core/docstring.py:56: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  doc = dedent(doc)
/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/src/flowcytometrytools/FlowCytometryTools/core/docstring.py:56: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  doc = dedent(doc)
/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib

In [2]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [10]:
flow_data, plate_properties = bld.write_flow_data_and_metadata(run_id, tx_config, aliquot_map_technique[run_id], work_dir=work_dir)

Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox) Plate 12
Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox) Plate 1
Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox) Plate 4
Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox) Plate 5
Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox) Plate 7
Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox) Plate 8
Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox) Plate 9
Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox) Plate 11
Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox) Plate 2
Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox) Plate 3
Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox) Plate 6
Attempting to fetch 16 datasets...
Getting data for container: Flow (Sytox

In [12]:
for p in plate_properties:
    df = bld.get_container_data(work_dir, p['container'])
    time_point = flow_data.loc[flow_data['container_name'] == p['container']]['time_point'].iloc[0]
    dead_volumes = list(df.kill_volume.dropna().unique())
    dead_volumes.remove(0.0)
    #print(dead_volumes)
    blda.train_models_for_stats(df, 
                            data_dir=work_dir, 
                            dead_volumes=dead_volumes,
                            fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'],
                           time_point=time_point)



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 140.0, 'stain': 'SYTOX Red Stain', 'time_point': '12'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 140.0, 'stain': nan, 'time_point': '12'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 210.0, 'stain': 'SYTOX Red Stain', 'time_point': '12'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 210.0, 'stain': nan, 'time_point': '12'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 280.0, 'stain': 'SYTOX Red Stain', 'time_point': '12'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 280.0, 'stain': nan, 'time_point': '12'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 1120.0, 'stain': 'SYTOX Red Stain', 'time_point': '12'}
{'experiment_id': 'r1dmsru

{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 2, 'live_volume': 0.0, 'dead_volume': 280.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 2, 'live_volume': 0.0, 'dead_volume': 280.0, 'stain': nan, 'time_point': '1'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 2, 'live_volume': 0.0, 'dead_volume': 1120.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 2, 'live_volume': 0.0, 'dead_volume': 1120.0, 'stain': nan, 'time_point': '1'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 3, 'live_volume': 0.0, 'dead_volume': 140.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 3, 'live_volume': 0.0, 'dead_volume': 140.0, 'stain': nan, 'time_point': '1'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 3, 'live_volume': 0.0, 'dead_volume': 210.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dmsrursbqwu

{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 280.0, 'stain': nan, 'time_point': '5'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 280.0, 'stain': 'SYTOX Red Stain', 'time_point': '5'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 1120.0, 'stain': nan, 'time_point': '5'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 1120.0, 'stain': 'SYTOX Red Stain', 'time_point': '5'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 210.0, 'stain': nan, 'time_point': '5'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 210.0, 'stain': 'SYTOX Red Stain', 'time_point': '5'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 140.0, 'stain': nan, 'time_point': '5'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_st

{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 2, 'live_volume': 0.0, 'dead_volume': 280.0, 'stain': nan, 'time_point': '7'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 2, 'live_volume': 0.0, 'dead_volume': 280.0, 'stain': 'SYTOX Red Stain', 'time_point': '7'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 2, 'live_volume': 0.0, 'dead_volume': 140.0, 'stain': nan, 'time_point': '7'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 2, 'live_volume': 0.0, 'dead_volume': 140.0, 'stain': 'SYTOX Red Stain', 'time_point': '7'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 3, 'live_volume': 0.0, 'dead_volume': 210.0, 'stain': nan, 'time_point': '7'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 3, 'live_volume': 0.0, 'dead_volume': 210.0, 'stain': 'SYTOX Red Stain', 'time_point': '7'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 3, 'live_volume': 0.0, 'dead_volume': 1120.0, 'stain': nan, 'time_point': '7'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_sta

{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 140.0, 'stain': 'SYTOX Red Stain', 'time_point': '9'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 140.0, 'stain': nan, 'time_point': '9'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 280.0, 'stain': 'SYTOX Red Stain', 'time_point': '9'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 280.0, 'stain': nan, 'time_point': '9'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 210.0, 'stain': 'SYTOX Red Stain', 'time_point': '9'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 210.0, 'stain': nan, 'time_point': '9'}
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 1120.0, 'stain': 'SYTOX Red Stain', 'time_point': '9'}
{'experiment_id': 'r1dmsrursbqwuz

KeyboardInterrupt: 

In [15]:
## Use last time point to predict live/dead for earlier time points

last_time = flow_data.loc[flow_data['time_point'].astype(int).idxmax()]
last_plate = [p for p in plate_properties if p['container'] == last_time['container_name']][0]
classifier_df = get_container_data(work_dir, last_plate['container'])
for p in plate_properties:
    df = get_container_data(work_dir, p['container'])
    time_point = flow_data.loc[flow_data['container_name'] == p['container']]['time_point'].iloc[0]
    dead_volumes = list(df.kill_volume.dropna().unique())
    dead_volumes.remove(0.0)

    pred_df = blda.train_models_for_prediction(classifier_df,
                                               df, 
                                               data_dir=work_dir,
                                 fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'],
                                               dead_strain_name=max(dead_volumes),
                                           combine_stains=False,
                                          overwrite=True,
                                          output_col = 'live',
                                              time_point=time_point)
    out_file = os.path.join(get_container_work_dir(work_dir, p['container']), 'data.csv')
    pred_df.to_csv(out_file)



output_col: live
SYTOX Red Stain
{'experiment_id': 'r1dmsrursbqwuz', 'random_state': 0, 'live_volume': 0, 'dead_volume': 1120.0, 'stain': 'SYTOX Red Stain', 'prediction': True, 'time_point': '12'}
data_columns: ['Unnamed: 0', 'checksum', 'filename', 'kill', 'kill_volume', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)



----------------------------------------------------------------------------------------------------
Starting run at time 15:45:42
Normalizing training and testing splits...
Starting Classification training...
Training time was: 45.93 seconds
Testing time was: 0.76 seconds
Prediction time of untested data was: 11.92351508140564

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                                           Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  6dL9DkeaOmlAW  2019-10-04  15:45:42  random_forest_classification

...


----------------------------------------------------------------------------------------------------
Starting run at time 16:11:27
Normalizing training and testing splits...
Starting Classification training...
Training time was: 24.10 seconds
Testing time was: 0.68 seconds
Prediction time of untested data was: 8.76300573348999

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                             Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  6D2PPrvl5DedW  2019-10-04  16:11:27  random_forest_classification  Hamed        

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)



----------------------------------------------------------------------------------------------------
Starting run at time 16:12:14
Normalizing training and testing splits...
Starting Classification training...
Training time was: 34.15 seconds
Testing time was: 0.87 seconds
Prediction time of untested data was: 8.81847882270813

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                                           Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  Q9r5Xy6W6Nxxe  2019-10-04  16:12:14  random_forest_classification 

In [64]:
# Single time point and Eth titration
blda.train_models_for_stats(df, 
                            data_dir=work_dir, 
                            fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'])


{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain'}
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 980.0, 'stain': None}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


HI
----------------------------------------------------------------------------------------------------
Starting run at time 22:31:18
Normalizing training and testing splits...
Starting Classification training...
Training time was: 40.89 seconds
Testing time was: 0.64 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                       Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  QgDA8YOg58WPP  2019-07-28  22:31:18  random_forest_classification  Hamed        2           0.994     0.994              1.0        1.0                0.994     0.993     

...

HI
----------------------------------------------------------------------------------------------------
Starting run at time 23:35:41
Normalizing training and testing splits...
Starting Classification training...
Training time was: 62.62 seconds
Testing time was: 1.57 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                 Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  5ZyJ9Xak7e2xw  2019-07-28  23:35:41  random_forest_classification  Hamed        2           0.647     0.647              0.709      0.677              0.673     

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


HI
----------------------------------------------------------------------------------------------------
Starting run at time 23:36:53
Normalizing training and testing splits...
Starting Classification training...
Training time was: 52.32 seconds
Testing time was: 1.77 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                    Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  52LjE9Q3LRJMM  2019-07-28  23:36:53  random_forest_classification  Hamed        2           0.638     0.638              0.691      0.673              0.64      0.637      0.

In [112]:
df = pd.read_csv(os.path.join(work_dir, 'data.csv'), index_col=0)

pred_df = blda.train_models_for_prediction(df, data_dir=work_dir,
                                 fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'],
                                           combine_stains=False,
                                          overwrite=True,
                                          output_col = 'live')
pred_df.to_csv(os.path.join(work_dir, 'data.csv'))

SYTOX Red Stain
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain', 'prediction': True}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


----------------------------------------------------------------------------------------------------
Starting run at time 14:03:08
Normalizing training and testing splits...
Starting Classification training...
Training time was: 41.69 seconds
Testing time was: 0.75 seconds
Prediction time of untested data was: 16.60473394393921

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                      Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  5W3jAjw9ez6zr  2019-07-30  14:03:08  random_forest_classification  Hamed        2      

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


----------------------------------------------------------------------------------------------------
Starting run at time 14:04:30
Normalizing training and testing splits...
Starting Classification training...
Training time was: 45.57 seconds
Testing time was: 0.85 seconds
Prediction time of untested data was: 16.545056104660034

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                         Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  EWoyeqNla8WWM  2019-07-30  14:04:30  random_forest_classification  Hamed        2           0.994  

In [116]:
# Train on just side scatter
df = pd.read_csv(os.path.join(work_dir, 'data.csv'), index_col=0)
pred_df = blda.train_models_for_prediction(df, data_dir=work_dir,
                                 fcs_columns=[ 'SSC-A','SSC-H', 'SSC-W'],
                                           combine_stains=False,
                                          overwrite=True,
                                          additional_description={
                                              'channels' : [ 'SSC-A','SSC-H', 'SSC-W']
                                          },
                                          output_col = 'live_ss')
pred_df.to_csv(os.path.join(work_dir, 'data.csv'))

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,2,3,4,5,6,7,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


SYTOX Red Stain
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain', 'prediction': True, 'channels': ['SSC-A', 'SSC-H', 'SSC-W']}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W', 'index', 'live']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['SSC-A', 'SSC-H', 'SSC-W']

----------------------------------------------------------------------------------------------------
Starting run at time 14:10:11
Normalizing training and testing splits...
Starting Classification training...
Training time was: 27.55 seconds
Testing time was: 0.84 seconds
Prediction time of untested data was: 17.089905261993408

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                                                               Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  6gD7RVbGQPB6N  2

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['SSC-A', 'SSC-H', 'SSC-W']

----------------------------------------------------------------------------------------------------
Starting run at time 14:11:17
Normalizing training and testing splits...
Starting Classification training...
Training time was: 28.10 seconds
Testing time was: 1.36 seconds
Prediction time of untested data was: 21.104886054992676

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                                                 Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  5RvNyJBOeZwrM  2019-07-30  14: